In [1]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast
from datetime import datetime

In [2]:
df = pd.read_csv(f"output/features_labeled_4llm_v3.csv")

In [3]:
df.head()

,abnormal_time,actual_moreoften,expected_moreoften,inject_pod,inject_time,inject_type,metrics_of_affectedpods,normal_time,root_cause,inject_comp_tdiff,nezha_rank_050_correct,nezha_rank_050_orig
0,2022-08-22 03:54,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,{'adservice-5f6585d649-fnmft': {'CpuUsageRate(...,2022-08-22 03:51,['CpuUsageRate(%)'],1,1,1
1,2022-08-22 03:55,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,{'adservice-5f6585d649-fnmft': {'CpuUsageRate(...,2022-08-22 03:51,['CpuUsageRate(%)'],2,1,1
2,2022-08-22 04:03,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...",frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,{'adservice-5f6585d649-fnmft': {'CpuUsageRate(...,2022-08-22 03:51,"['Serving product page started', 'GetProduct s...",1,2,2
3,2022-08-22 04:04,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...",frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,{'adservice-5f6585d649-fnmft': {'CpuUsageRate(...,2022-08-22 03:51,"['Serving product page started', 'GetProduct s...",2,2,2
4,2022-08-22 04:11,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",frontend-579b9bff58-t2dbm,2022-08-22 04:10:20,cpu_consumed,{'adservice-5f6585d649-fnmft': {'CpuUsageRate(...,2022-08-22 03:51,['CpuUsageRate(%)'],1,1,1


In [4]:
log_template_miner = get_miner()

In [5]:
df['actual_moreoften'] = df['actual_moreoften'].apply(lambda x: ast.literal_eval(x))
df['expected_moreoften'] = df['expected_moreoften'].apply(lambda x: ast.literal_eval(x))

In [6]:
def format_result_list(result_list):
    """
    Format result list for LLM comprehension
    """
    for i in range(len(result_list)):
        entry = result_list[i]
        if 'events' in entry:
            entry['events'] = [from_id_to_template(int(entry['events'].split('_')[0]), log_template_miner),
                                      from_id_to_template(int(entry['events'].split('_')[1]), log_template_miner)]
        if 'events_inreferenz' in entry:
            entry['events_inreferenz'] = [from_id_to_template(int(entry['events_inreferenz'].split('_')[0]), log_template_miner),
                                      from_id_to_template(int(entry['events_inreferenz'].split('_')[1]), log_template_miner)]
    
    return result_list

In [7]:
df['actual_moreoften'] = df['actual_moreoften'].apply(format_result_list)

In [8]:
df['expected_moreoften'] = df['expected_moreoften'].apply(format_result_list)

In [9]:
pd.set_option("display.max_colwidth", None)
df[['expected_moreoften']].head()

expected_moreoften
0                                                                                                                                                                                                                                                                             [{'events': ['frontend hipstershop.Frontend/Recv. start', 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Request started'], 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_inreferenz': ['frontend hipstershop.Frontend/Recv. start', 'CpuUsageRate(%)']}, {'events': ['currencyservice grpc.hipstershop.CurrencyService/Convert start', 'currencyservice hipstershop.CurrencyService/Carry start'], 'score': 0.5638418079096045, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_inreferenz': ['currencyservice grpc.hipstershop.CurrencyService/Convert start', 'currencyservice hipstershop.CurrencyService/GetCurrencyData start']}, {'events': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Received conversion request', 'currencyservice hipstershop.CurrencyService/GetCurrencyData start'], 'score': 0.52, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_inreferenz': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Received conversion request', 'currencyservice hipstershop.CurrencyService/Carry start']}, {'events': ['MemoryUsageRate(%)', 'adservice hipstershop.AdService/<:SPANID:> start'], 'score': 0.514161220043573, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'events_inreferenz': ['MemoryUsageRate(%)', 'adservice hipstershop.AdService/getRandomAds start']}, {'events': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>', 'adservice hipstershop.AdServiceImpl/getAds end'], 'score': 0.514161220043573, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'events_inreferenz': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>', '<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> No context provided. Constructe random Ads start']}, {'events': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Choose Ad complete', 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Serving product page complete'], 'score': 0.514161220043573, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_inreferenz': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Choose Ad complete', 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Request home complete']}, {'events': ['adservice hipstershop.AdService/<:SPANID:> start', 'MemoryUsageRate(%)'], 'score': 0.5140495867768595, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft'}, {'events': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Query items successfully.', '<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Get Ads by Category <:*:>'], 'score': 0.5140495867768595, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'events_inreferenz': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Query items successfully.', 'adservice hipstershop.AdService/getRandomAds end']}, {'events': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>', 'adservice hipstershop.AdService/<:SPANID:> start'], 'score': 0.5136986301369864, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'events_inreferenz': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>', '<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> No context provided. Constructe random Ads start']}, {'events': ['currencyservice grpc.hipstershop.CurrencyService/GetSupportedCurrencies start', 'currencyservice hipstershop.CurrencyService/GetCurrencyData start'], 'score': 0.5100250626566416, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd'}]
1                                    

In [10]:
df.columns

Index(['abnormal_time', 'actual_moreoften', 'expected_moreoften', 'inject_pod',
       'inject_time', 'inject_type', 'metrics_of_affectedpods', 'normal_time',
       'root_cause', 'inject_comp_tdiff', 'nezha_rank_050_correct',
       'nezha_rank_050_orig'],
      dtype='object')

In [11]:
df.inject_pod.unique()

array(['frontend-579b9bff58-t2dbm', 'cartservice-579f59597d-wc2lz',
       'checkoutservice-578fcf4766-9csqn',
       'currencyservice-cf787dd48-vpjrd', 'emailservice-55fdc5b988-f6xth',
       'paymentservice-9cdb6588f-554sm',
       'productcatalogservice-668d5f85fb-wckp8',
       'recommendationservice-6cfdd55578-gfj6q',
       'shippingservice-7b598fc7d-lmggd', 'adservice-5f6585d649-fnmft',
       'productcatalogservice-668d5f85fb-jhwx9'], dtype=object)

In [12]:
df.to_csv(f"output/features_labeled_4llm_v3_step2.csv", index=False)